ЗАДАЧА:
сделать бесконечй цикл для диалога с моделью в одном чате с сохранением всего этого контекста:
+ если пользователь вводит "/exit", то выйти из цикла
+ если вводит "/clear", то очистить историю, кроме system prompt
+ если вводит "/system какой-то текст", то всё что после /system - будет системный промптом
+ для задачи обязательно использовать rich console https://rich.readthedocs.io/en/latest/console.html
+ и panel для красоты https://rich.readthedocs.io/en/latest/panel.html
+ форматирование через https://rich.readthedocs.io/en/latest/markdown.html

In [220]:
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich import box

from openai import OpenAI

from enum import Enum

from pydantic import SecretStr, HttpUrl, BaseModel
from pydantic_settings import SettingsConfigDict, BaseSettings

import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

True

In [221]:
console = Console(color_system="standard")

In [ ]:
class OpenAISettings(BaseSettings):
  
  openai_api_key: SecretStr
  openai_base_url: HttpUrl
  
  model: str = "GigaChat/GigaChat-2-Max"
  
  model_config = SettingsConfigDict(env_file=".env")
  
configs = OpenAISettings()

In [223]:
class RoleModel(Enum):
  SYSTEM = "system"
  USER = "user"
  ASSISTANT = "assistant"
class MessageModel(BaseModel):
  role: str
  content: str

In [224]:
def display_message_with_formatting(*, role: RoleModel, content: str) -> None:
  
      if role == "user":
        style = "bold blue"
        border_style = "blue"
      elif role == "assistant":
          style = "bold green"
          border_style = "green"
          content = Markdown(content)
      elif role == "system":
          style = "bold yellow"
          border_style = "yellow"
      else:
          style = ""
          border_style = "white"

      console.print(
          Panel(
              content,
              title=f"[{style}]{role}",
              title_align="center",
              border_style=border_style,
              box=box.ROUNDED,
              expand=False,
          )
      )

In [225]:
def get_client() -> OpenAI:
    settings = OpenAISettings()
    client = OpenAI(
        api_key=settings.openai_api_key.get_secret_value(),
        base_url=str(settings.openai_base_url),
    )
    return client

In [226]:
def chat_completion(*, model: str, messages: list) -> list[MessageModel]:
  client = get_client()
  
  completion = client.chat.completions.create(
    model=model, 
    temperature=1, 
    max_tokens=2000, 
    messages=messages
  )
  
  content = completion.choices[0].message.content.strip()
  new_message = MessageModel(role=RoleModel.ASSISTANT, content=content)
  
  display_message_with_formatting(role=RoleModel.ASSISTANT.value, content=content)

  messages.append(new_message.model_dump())
  
  return messages

In [227]:
def handle_user_input(*, user_input: str, messages: list[MessageModel]) -> tuple[list[MessageModel], bool]:

  if user_input.lower().startswith("/exit"):
    console.print(Panel("Exit from loop - /exit command", style="red", expand=False))
    return [], False
  elif user_input.lower().startswith("/clear"):
    console.print(Panel("Clear all messages history - /clear command", style="yellow", expand=False))
    messages = [message for message in messages if message["role"] == "system"]
  elif user_input.lower().startswith("/system"):
    console.print(Panel("Set new system prompt - /system command", style="yellow", expand=False))
    system_prompt = user_input.replace("/system", "").strip()
    system_message = MessageModel(role="system", content=system_prompt)
    if not len(messages):
      messages.append(system_message.model_dump())
    else:
      for index, message in enumerate(messages):
        if message["role"] == "system":
          messages[index] = system_message.model_dump()
  else:
    console.print(Panel("Add new message in messages history", style="green", expand=False))
    new_message = MessageModel(role=RoleModel.USER, content=user_input)
    messages.append(new_message.model_dump())
    
  return messages, True

In [228]:
def start_infinity_chat() -> None:
  
  messages = []

  while True:
    user_input = str(input("Введите входные данные для продолжения общения с ИИ...: ")).strip()
    
    (messages, is_should_continue) = handle_user_input(user_input=user_input, messages=messages)
    
    if not is_should_continue:
      break
    
    messages = chat_completion(model=configs.model, messages=messages)
    
start_infinity_chat()

╭─────────────────────────────────────────╮
│ Set new system prompt - /system command │
╰─────────────────────────────────────────╯

╭─────────────────────────────────────────────────── assistant ───────────────────────────────────────────────────╮
│ Задавай вопросы — разберёмся вместе!                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────╮
│ Add new message in messages history │
╰─────────────────────────────────────╯

╭─────────────────────────────────────────────────── assistant ───────────────────────────────────────────────────╮
│ Параллельная парковка — одна из важнейших маневров для водителя любого большого города вроде Ростова-на-Дону,   │
│ где плотность машин высока, места ограничены, а хорошие парковочные зоны часто встречаются редко. Правильная    │
│ параллельная парковка требует внимательности, спокойствия и уверенности в действиях.                            │
│                                                                                                                 │
│                                              Инструкция пошагово:                                               │
│                                                                                                                 │
│  1 Выберите место                                                                                               │
│    Перед началом парковки убедитесь, что выбранное пространство достаточно велико: оно должно превышать длину   │
│    вашего автомобиля примерно на 1–1,5 метра спереди и сзади.                                                   │
│  2 Подготовьтесь к началу манёвра Выровняйте машину параллельно автомобилю впереди выбранного вами свободного   │
│    пространства, остановитесь на расстоянии около полуметра от него бок о бок. Важно, чтобы ваши задние бамперы │
│    были примерно на одном уровне.                                                                               │
│  3 Начало движения назад Включите правый поворотник, проверьте зеркала заднего вида и обстановку вокруг себя.   │
│    Убедившись, что всё чисто, начните движение назад медленно и аккуратно.                                      │
│  4 Первая точка поворота Когда ваш задний угол машины достигнет уровня начала передней фары соседнего авто      │
│    позади вас, поверните руль вправо до упора. Медленно продолжайте двигаться назад.                            │
│  5 Корректировка положения Как только ваша машина окажется примерно под углом 45 градусов относительно бордюра, │
│    быстро выровняйте руль прямо и двигайтесь дальше назад.                                                      │
│  6 Вторая точка поворота Дождитесь момента, когда передний край вашей машины будет примерно напротив середины   │
│    стоящего рядом автомобиля (спереди), тогда резко выкрутите руль влево до конца.                              │
│  7 Завершение парковки Продолжая ехать назад, следите за расстоянием до обоих соседних автомобилей.             │
│    Остановитесь, когда вы ровно встали между ними, либо расстояние от переднего и заднего края одинаково.       │
│  8 Поставьте автомобиль в нужное положение Если нужно немного откорректировать позицию (подвинуть ближе к       │
│    бордюру или чуть вперёд/назад), используйте медленные аккуратные передвижения вперед-назад.                  │
│                                                                                                                 │
│ ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── │
│                                                 Важные советы:                                                  │
│                                                                                                                 │
│  • Внимательно смотрите в боковые зеркала во время выполнения всех действий!                                    │
│  • Не спешите: лучше сделать этот процесс дольше, но корректнее и увереннее.                                    │
│  • Следите за пешеходами, велосипедистами и машинами поблизости — городской трафик непредсказуем.               │
│  • Регулярно проверяйте дистанцию до окружающих объектов.                                                       │
│  • Используйте короткие расстояния для небольших движений — чрезмерная скорость здесь неуместна.                │
│                                                       

╭─────────────────────────────────────╮
│ Add new message in messages history │
╰─────────────────────────────────────╯

...

╭─────────────────────────────────────────╮
│ Set new system prompt - /system command │
╰─────────────────────────────────────────╯

╭─────────────────────────────────────────────────── assistant ───────────────────────────────────────────────────╮
│ Объясню понятнее и нагляднее.                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────╮
│ Add new message in messages history │
╰─────────────────────────────────────╯

╭─────────────────────────────────────────────────── assistant ───────────────────────────────────────────────────╮
│ Эластичные (упругие) и жёсткие муфты отличаются друг от друга конструктивно и имеют разные преимущества и       │
│ недостатки в зависимости от условий эксплуатации техники.                                                       │
│                                                                                                                 │
│                                                Эластичная муфта:                                                │
│                                                                                                                 │
│  • Преимущества:                                                                                                │
│     • Поглощение вибраций и ударов. Благодаря упругости демпферных элементов такие муфты снижают нагрузку на    │
│       двигатель и трансмиссию, продлевая срок службы механизмов.                                                │
│     • Защита оборудования от перегрузок и аварийных ситуаций. Например, если случайно произошел удар агрегата   │
│       или застрял вал отбора мощности (ВОМ), эластичность муфты позволяет предотвратить повреждение             │
│       дорогостоящих узлов.                                                                                      │
│     • Меньший износ деталей двигателя и трансмиссии благодаря смягчению крутильных колебаний и неравномерности  │
│       вращения.                                                                                                 │
│  • Недостатки:                                                                                                  │
│     • Уменьшение КПД передачи момента, особенно при больших нагрузках и интенсивной работе.                     │
│     • Ограниченная передаваемая мощность по сравнению с жесткой муфтой аналогичного размера.                    │
│     • Необходимость периодической замены резиновых или полиуретановых элементов вследствие износа.              │
│                                                                                                                 │
│                                                 Жесткая муфта:                                                  │
│                                                                                                                 │
│  • Преимущества:                                                                                                │
│     • Максимально полное использование мощности двигателя. За счет прямой передачи вращающего момента потери    │
│       минимальны.                                                                                               │
│     • Надежность и долговечность конструкции. Простота устройства обеспечивает длительный ресурс работы без     │
│       частых ремонтов.                                                                                          │
│     • Способность передавать большие нагрузки и выдерживать высокие скорости вращения.                          │
│  • Недостатки:                                                                                                  │
│     • Передача всех вибраций и толчков непосредственно на механизм и детали трактора/комбайна, что увеличивает  │
│       риск повреждения важных компонентов при длительных тяжелых условиях эксплуатации.                         │
│     • Отсутствие защиты от критичных перегрузок. В случае заклинивания рабочих органов (например, ножа жатки,   │
│       зубьев культиватора) жесткий тип соединения приведет к поломке дорогостоящего узла или даже разрушению    │
│       основных агрегатов.                                                                                       │
│                                                                                                                 │
│ ──────────────────────────────────────────────────────

╭────────────────────────────────╮
│ Exit from loop - /exit command │
╰────────────────────────────────╯